In [1]:
import pandas as pd
import os
location = str(os.getcwd())

In [2]:
f = (location[:-8] + "Articles\\articles_cleaned.csv")
articles = pd.read_csv(f)
articles = articles.to_dict("index")

In [3]:
f = (location[:-8] + "Tweets\\tweets_cleaned.csv")
tweets = pd.read_csv(f)
tweets = tweets.to_dict("index")

In [4]:
locations = ["IUB", "UMD", "UMN", "Stanford", "Wisconsin"]
dates = []
for i in range(len(tweets)):
    tweet = tweets[i]
    if tweet["created_at"] not in dates:
        dates.append(tweet["created_at"])
for i in range(len(articles)):
    article = articles[i]
    if article["date"] not in dates:
        dates.append(article["date"])
print(len(dates))

4840


In [6]:
by_date = []

for location in locations:
    for date in dates:
        date_dict = {}
        date_dict["location"] = location
        date_dict["date"] = date
        tweet_count = 0
        article_count = 0

        for i in range(len(tweets)):
            tweet = tweets[i]
            if date == tweet["created_at"] and tweet["location"] == location:
                tweet_count += 1

        for j in range(len(articles)):
            article = articles[j]
            if date == article["date"] and article["location"] == location:
                article_count += 1

        date_dict["tweet_count"] = tweet_count
        date_dict["article_count"] = article_count

        by_date.append(date_dict)

In [8]:
pd.DataFrame(by_date).to_csv('counts_by_day.csv')